# Imports and Installs

In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=42dd92d4615e00eeb61fc8857db46ca3e32f6827f9800eb1024f937656fbb916
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [2]:
# import standard PyTorch modules
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # TensorBoard support
import torch.profiler
# import torchvision module to handle image manipulation
import torchvision
from torchvision import datasets 
import torchvision.transforms as transforms
import torchvision.models as models
from torchsummary import summary
from tqdm import tqdm
# calculate train time, writing train data to files etc.
import time
import pandas as pd
import json
import math
import copy
from IPython.display import clear_output
from time import perf_counter
from torch.autograd import Variable
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)     # On by default, leave it here for clarity

In [4]:
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split

In [5]:
import matplotlib.pyplot as plt
import seaborn

In [6]:
import matplotlib.image as mpimg

In [7]:
# check PyTorch versions
print(torch.__version__)
print(torchvision.__version__)

2.0.0+cu118
0.15.1+cu118


In [8]:
import wandb

# Logging into WandB


In [9]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Loading Dataset


In [10]:
from torchvision import transforms
from torchvision.transforms import ToTensor, Normalize

train_transforms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [11]:
dataset_train = torchvision.datasets.CIFAR10('./data', download=True, train=True, transform=train_transforms)

# Download and load the test set
dataset_test = torchvision.datasets.CIFAR10('./data', download=True, train=False, transform=test_transforms)

100%|██████████| 170498071/170498071 [00:03<00:00, 56564444.51it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [20]:
trainset, valset = torch.utils.data.random_split(dataset_train, [40000, 10000])
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(dataset_test, batch_size=64, shuffle=False)

# set cuda to device

In [13]:
#torch.cuda.memory_summary(device=None, abbreviated=True)

In [14]:
torch.cuda.empty_cache()

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#Some key definitions

## Count of correct predictions

In [16]:
def log_test_predictions(images, labels, outputs, predicted, test_table, log_counter):
  # obtain confidence scores for all classes
  scores = F.softmax(outputs.data, dim=1)
  log_scores = scores.cpu().numpy()
  log_images = images.cpu().numpy()
  log_labels = labels.cpu().numpy()
  log_preds = predicted.cpu().numpy()
  # adding ids based on the order of the images
  _id = 0
  for i, l, p, s in zip(log_images, log_labels, log_preds, log_scores):
    # add required info to data table:
    # id, image pixels, model's guess, true label, scores for all classes
    img_id = str(_id) + "_" + str(log_counter)
    test_table.add_data(img_id, wandb.Image(np.transpose(i, (1, 2, 0))), p, l, *s)
    _id += 1
    if _id == NUM_IMAGES_PER_BATCH:
      break


In [17]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [18]:
def get_topk_correct(preds,labels,k=5):
    batch_size = preds.shape[0]

    # get the top-5 predictions for each sample
    topk_values, topk_indices = torch.topk(preds, k=5, dim=1)

    # check if the target is in the top-5 predictions
    correct_topk = topk_indices.eq(labels.view(-1, 1).expand_as(topk_indices))
    return correct_topk
## calculate top-5 accuracy
#top5_accuracy = correct_topk.float().sum() / batch_size

## taking value of parameters and trying them in all combinations

In [19]:
# import modules to build RunBuilder and RunManager helper classes
from collections  import OrderedDict
from collections import namedtuple
from itertools import product

# Read in the hyper-parameters and return a Run namedtuple containing all the 
# combinations of hyper-parameters
class RunBuilder():
  @staticmethod
  def get_runs(params):

    Run = namedtuple('Run', params.keys())

    runs = []
    for v in product(*params.values()):
      runs.append(Run(*v))
    
    return runs

## defining Manager class 

This helps in calculating both train and validation accuracy and losses also help us to maintain a tensorboard and print all the data collected for every epoch and save it in a dataset 

In [34]:
# Helper class, help track loss, accuracy, epoch time, run time, 
# hyper-parameters etc. Also record to TensorBoard and write into csv, json
class RunManager():
  def __init__(self):

    # tracking every epoch count, loss, accuracy, time
    self.epoch_count = 0
    self.epoch_loss = 0
    self.epoch_val_loss = 0
    self.epoch_num_correct_train = 0
    self.epoch_num_correct_test = 0
    self.epoch_start_time = None

    # tracking every run count, run data, hyper-params used, time
    self.run_params = None
    self.run_count = 0
    self.run_data = []
    self.run_start_time = None

    # record model, loader and TensorBoard 
    self.network = None
    self.trainloader = None
    self.valloader = None
    self.tb = None

  # record the count, hyper-param, model, loader of each run
  # record sample images and network graph to TensorBoard  
  def begin_run(self, run, network, trainloader,valloader):

    self.run_start_time = time.time()

    self.run_params = run
    self.run_count += 1

    self.network = network
    self.trainloader = trainloader
    self.valloader = valloader
    self.tb = SummaryWriter(comment=f'-{run}')

    wandb.init(
        # set the wandb project where this run will be logged
        project="dl_project_2023",
        
        # track hyperparameters and run metadata
        config={
        "learning_rate": run.lr,
        "architecture": run.arch,
        "dataset": run.dataset,
        "epochs": 30,
        "activation_function":run.act_func,
        "Optimizer":run.opt,
        "Regularisation":run.reg
        }
    )
    #images, labels = next(iter(self.trainloader))
    #images, labels = images.to(device), labels.to(device)
    #grid = torchvision.utils.make_grid(images)

    #self.tb.add_image('images', grid)
    #self.tb.add_graph(self.network, images)

  # when run ends, close TensorBoard, zero epoch count
  def end_run(self):
    wandb.finish()
    self.tb.close()
    self.epoch_count = 0

  # zero epoch count, loss, accuracy, 
  def begin_epoch(self):
    self.epoch_start_time = time.time()

    self.epoch_count += 1
    self.epoch_loss = 0
    self.epoch_val_loss = 0
    self.epoch_num_correct_train = 0
    self.epoch_num_correct_test = 0
    print("Training Epoch",self.epoch_count)

  # 
  def end_epoch(self):
    # calculate epoch duration and run duration(accumulate)
    epoch_duration = time.time() - self.epoch_start_time
    run_duration = time.time() - self.run_start_time

    # record epoch loss and accuracy
    train_loss = self.epoch_loss / len(self.trainloader.dataset)
    val_loss = self.epoch_val_loss / len(self.valloader.dataset)
    train_accuracy = self.epoch_num_correct_train / len(self.trainloader.dataset)
    test_accuracy = self.epoch_num_correct_test / len(self.valloader.dataset)
    # Record epoch loss and accuracy to TensorBoard 
    self.tb.add_scalar('Train Loss', train_loss, self.epoch_count)
    self.tb.add_scalar('train Accuracy', train_accuracy, self.epoch_count)
    self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
    self.tb.add_scalar('Validation Accuracy', test_accuracy, self.epoch_count)

    # Record params to TensorBoard
    #for name, param in self.network.named_parameters():
      #self.tb.add_histogram(name, param, self.epoch_count)
      #self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
    
    # Write into 'results' (OrderedDict) for all run related data
    results = OrderedDict()
    results["run"] = self.run_count
    results["epoch"] = self.epoch_count
    results["train_loss"] = train_loss
    results["val_loss"] = val_loss
    results["train_accuracy"] = train_accuracy
    results["val_accuracy"] = test_accuracy
    results["epoch duration"] = epoch_duration
    results["run duration"] = run_duration
    print("train_acc = ", train_accuracy, "train_loss = ", train_loss, "Validation_acc = ", test_accuracy, "Validation_loss = ", val_loss)
    wandb.log({"train_acc": train_accuracy, "train_loss": train_loss, "Validation_acc": test_accuracy, "Validation_loss": val_loss})
    
    # Record hyper-params into 'results'
    for k,v in self.run_params._asdict().items(): results[k] = v
    self.run_data.append(results)
    df = pd.DataFrame.from_dict(self.run_data, orient = 'columns')

    # display epoch information and show progress
    # clear_output(wait=True)
    # display(df)

  # accumulate loss of batch into entire epoch loss
  def track_loss(self, loss):
    # multiply batch size so variety of batch sizes can be compared
    self.epoch_loss += loss.item() * self.trainloader.batch_size
  def track_val_loss(self, loss):
    # multiply batch size so variety of batch sizes can be compared
    self.epoch_val_loss += loss.item() * self.valloader.batch_size
  # accumulate number of corrects of batch into entire epoch num_correct
  def track_num_correct_train(self, preds, labels):
    self.epoch_num_correct_train += self._get_num_correct(preds, labels)
  def track_num_correct_test(self, preds, labels):
    self.epoch_num_correct_test += self._get_num_correct(preds, labels)
  @torch.no_grad()
  def _get_num_correct(self, preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()
  
  # save end results of all runs into csv, json for further a
  def save(self, fileName):

    pd.DataFrame.from_dict(
        self.run_data, 
        orient = 'columns',
    ).to_csv(f'{fileName}.csv')

    with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
      json.dump(self.run_data, f, ensure_ascii=False, indent=4)

# Defining Shake Drop Regularisation


In [22]:
class ShakeDropFunction(torch.autograd.Function):

    @staticmethod
    def forward(ctx, x, training=True, p_drop=0.5, alpha_range=[-1, 1]):
        if training:
            gate = torch.cuda.FloatTensor([0]).bernoulli_(1 - p_drop)
            ctx.save_for_backward(gate)
            if gate.item() == 0:
                alpha = torch.cuda.FloatTensor(x.size(0)).uniform_(*alpha_range)
                alpha = alpha.view(alpha.size(0), 1, 1, 1).expand_as(x)
                return alpha * x
            else:
                return x
        else:
            return (1 - p_drop) * x

    @staticmethod
    def backward(ctx, grad_output):
        gate = ctx.saved_tensors[0]
        if gate.item() == 0:
            beta = torch.cuda.FloatTensor(grad_output.size(0)).uniform_(0, 1)
            beta = beta.view(beta.size(0), 1, 1, 1).expand_as(grad_output)
            beta = Variable(beta)
            return beta * grad_output, None, None, None
        else:
            return grad_output, None, None, None


class ShakeDrop(nn.Module):

    def __init__(self, p_drop=0.5, alpha_range=[-1, 1]):
        super(ShakeDrop, self).__init__()
        self.p_drop = p_drop
        self.alpha_range = alpha_range

    def forward(self, x):
        return ShakeDropFunction.apply(x, self.training, self.p_drop, self.alpha_range)

# Building Network as per given Instructions

In [24]:
def define_model(reg):
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
    # model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(512, 10)
    if reg == 'shake-drop':
        # model.relu = nn.Sequential(ShakeDrop(0.5),
        #                             nn.ReLU())
        # print(model.relu)
        model.layer1[0].conv1 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
                                            ShakeDrop(0.2))
        model.layer2[0].conv1 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
                                            ShakeDrop(0.2))
        model.layer3[0].conv1 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
                                            ShakeDrop(0.2))
        model.layer4[0].conv1 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
                                            ShakeDrop(0.2))
        return model
    elif reg == 'dropout':
        # model.relu = nn.Sequential(nn.Dropout(p=0.2),
        #                             nn.ReLU())
        # print(model.relu)
        model.layer1[0].conv1 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
                                            nn.Dropout(p=0.2))
        model.layer2[0].conv1 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
                                            nn.Dropout(p=0.2))
        model.layer3[0].conv1 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
                                            nn.Dropout(p=0.2))
        model.layer4[0].conv1 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
                                            nn.Dropout(p=0.2))
        return model
    else:
        return model

In [27]:
model = define_model('shake-drop').to(device)
summary(model, ( 3, 32, 32))

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
         ShakeDrop-6             [-1, 64, 8, 8]               0
       BatchNorm2d-7             [-1, 64, 8, 8]             128
              ReLU-8             [-1, 64, 8, 8]               0
            Conv2d-9             [-1, 64, 8, 8]          36,864
      BatchNorm2d-10             [-1, 64, 8, 8]             128
             ReLU-11             [-1, 64, 8, 8]               0
       BasicBlock-12             [-1, 64, 8, 8]               0
           Conv2d-13             [-1, 64, 8, 8]          36,864
      BatchNorm2d-14             [-1, 6

# Training the Model

we write the code to train our network 

In [32]:
params = OrderedDict(
    lr = [.001],
    batch_size = [64],
    shuffle = [True],
    arch = ["Resnet18"],
    dataset = ["Cifar10"],
    act_func = ["ReLU"],
    opt = ["ADAM"],
    reg = ['shake-drop','dropout','no_reg']

)
epochs = 30
# Number of batches to log from the test data for each test step
# (default set low to simplify demo)
NUM_BATCHES_TO_LOG = 10 #79

# Number of images to log per test batch
# (default set low to simplify demo)
NUM_IMAGES_PER_BATCH = 32 #128

In [35]:
m = RunManager()



for run in RunBuilder.get_runs(params):
    
    model = define_model(run.reg)
    model = model.to(device)
    
    if run.act_func == "Tanh":
        model.relu = nn.Tanh()
    
    if run.opt == "ADAM":
        optimizer = optim.Adam(model.parameters(), lr=run.lr)
    elif run.opt == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=run.lr)
    
    m.begin_run(run, model, train_loader,val_loader)
    for epoch in range(epochs):
        
        m.begin_epoch()
        batch_count = 0
        for batch in tqdm(train_loader):
            images = batch[0]
            labels = batch[1]
            images, labels = images.to(device), labels.to(device)
            preds = model(images)
            loss = F.cross_entropy(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            m.track_loss(loss)
            m.track_num_correct_train(preds, labels)
        for batch in tqdm(val_loader):
        
            images = batch[0]
            labels = batch[1]
            images, labels = images.to(device), labels.to(device)
            preds = model(images)
            loss = F.cross_entropy(preds, labels)
            m.track_val_loss(loss)
            m.track_num_correct_test(preds, labels)
        
        m.end_epoch()
    columns=["id", "image", "guess", "truth"]
    for digit in range(10):
      columns.append("score_" + str(digit))
    test_table = wandb.Table(columns=columns)
    log_counter = 0
    sum = 0
    for batch in test_loader:
            
        images = batch[0]
        labels = batch[1]
        images, labels = images.to(device), labels.to(device)
        preds = model(images)
        _, predicted = torch.max(preds.data, 1)
        #loss = F.cross_entropy(preds, labels)

        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()
        if log_counter < NUM_BATCHES_TO_LOG:
            log_test_predictions(images, labels, preds, predicted, test_table, log_counter)
            log_counter += 1
        #m.track_val_loss(loss)
        sum += get_num_correct(preds, labels)
    print('Test Accuracy is ',sum/len(test_loader.dataset))
    wandb.log({"test_predictions" : test_table})
    m.end_run()

# when all runs are done, save results to files
m.save('results_resnet_18')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Validation_acc,▁▅▆██
Validation_loss,█▅▄▁▁
train_acc,▁▄▆▇█
train_loss,█▅▃▂▁
Validation_acc,0.6644
Validation_loss,0.94792
train_acc,0.68815
train_loss,0.89499


Training Epoch 1


100%|██████████| 157/157 [00:06<00:00, 25.02it/s]


train_acc =  0.42025 train_loss =  1.592890866470337 Validation_acc =  0.4957 Validation_loss =  1.4101313358306884
Training Epoch 2


...

100%|██████████| 157/157 [00:05<00:00, 28.90it/s]


train_acc =  0.9328 train_loss =  0.190731995177269 Validation_acc =  0.7689 Validation_loss =  0.8925747182846069
Training Epoch 30


100%|██████████| 157/157 [00:05<00:00, 26.79it/s]


train_acc =  0.9377 train_loss =  0.17771539914309978 Validation_acc =  0.7721 Validation_loss =  0.8998922765731812
Test Accuracy is  0.7942


Validation_acc,▁▃▄▅▅▆▆▆▇▇▇▇▇█▇▇██████▇███████
Validation_loss,█▆▅▃▃▂▂▂▁▁▁▁▁▁▁▂▂▁▂▁▂▂▃▂▂▂▃▃▃▃
train_acc,▁▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█████████
train_loss,█▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Validation_acc,0.7721
Validation_loss,0.89989
train_acc,0.9377
train_loss,0.17772
